姓名：于永夏
学号：2022201050177

利用AG_News数据库，通过输入相应文字来判别是哪个类型的文本
类型分为四类"World" "Sports" "Business" "Sci/Tec"
input文件夹 是存放输入数据库
外部的test.csv和train.csv 是新生成的文件

In [1]:
# 运算b步骤如下

"""
1. process the ag_news dataset
2. define the model of classification
3. train the model to classification the ag_news
4. test the model
"""

'\n1. process the ag_news dataset\n2. define the model of classification\n3. train the model to classification the ag_news\n4. test the model\n'

1. process the ag_news dataset

In [2]:
import pandas as pd
import pathlib
import numpy as np
import torch

In [3]:
from ast import Index


class MyTextDataset(torch.utils.data.IterableDataset):
    def __init__(self, csv_path, train=True):
        super(MyTextDataset, self).__init__()
        name = 'train' if train else 'test'
        self.data_file = pd.read_csv(csv_path, encoding='utf-8', sep=',')
        self.labels = list(self.data_file["label"])
        self.texts = list(self.data_file["text"])
        print(f"the {name} dataset are {len(self.data_file.index)}'s numbers")

    def __iter__(self):
        return iter(zip(self.labels, self.texts))

    def __next__(self):
        return next(iter(zip(self.labels, self.texts)))

    def __len__(self):
        return len(self.data_file.index)


def convert_dataset(raw_path: str, new_path: str, train: bool = True):
    """
    convert the ag_news train.csv or test.csv(contain 3 columns:Class Index,Title,Description) \
    generate a new train.csv or test.csv(contain 2 columns:Class Index,Description)
    notice!! the Description contains the Title

    args:
    - raw_path: input a dataset path
    - new_path: save new train.csv or test.csv into here
    """

    name = 'train' if train else 'test'
    raw_path = pathlib.Path(raw_path)
    raw_path = list(raw_path.glob(f"*{name}.csv"))[0]
    raw_path = str(raw_path)

    raw_file = pd.read_csv(raw_path)
    # print(len(raw_file.index))
    rows = raw_file.shape[0]
    columns = raw_file.shape[1] - 1

    new_path = new_path + f"/{name}.csv"
    # print(new_path)
    new_file = pd.DataFrame(np.zeros((rows, columns)), index=np.arange(rows), columns=["label", "text"])
    new_file["label"] = raw_file["Class Index"]
    new_file["text"] = raw_file["Title"]
    new_file["text"] += raw_file["Description"]

    # x = raw_file["Title"] + raw_file["Description"]
    # print(x)
    new_file.to_csv(new_path)

In [4]:
file_path = "./input"
destination = "./"
convert_dataset(file_path, destination, train=True)
convert_dataset(file_path, destination, train=False)
train_dataset = MyTextDataset("./train.csv")
print(next(train_dataset))

the train dataset are 120000's numbers
(3, "Wall St. Bears Claw Back Into the Black (Reuters)Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")


2. define the model of classification

In [5]:
import torch.nn as nn
import torch

barch_size = 64

In [7]:
class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.num_layers = 2
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
#         self.lstm = nn.LSTM(embed_dim, 64, self.num_layers, bidirectional=True)
        self.gru = nn.GRU(embed_dim, 64, self.num_layers, bidirectional=True)
        self.fc = nn.Linear(128, num_class)
        self.init_weights()
#         # lstm parameters
#         self.h0 = torch.randn(2*self.num_layers, barch_size, 128)
#         self.c0 = torch.randn(2*self.num_layers, barch_size, 128)
#         # gru parameters
#         self.h1 = torch.randn(2*self.num_layers, barch_size, 256)
#         self.c1 = torch.randn(2*self.num_layers, barch_size, 256)

    def init_weights(self):
        init_range = 0.5
        self.embedding.weight.data.uniform_(-init_range, init_range)
        self.fc.weight.data.uniform_(-init_range, init_range)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        x = self.embedding(text, offsets)
#         x = self.lstm(x, (self.h0,self.c0))
#         x = self.gru(x, (self.h1,self.c1))
#         x,(_h,_c) = self.lstm(x)
        x,_h = self.gru(x)
        x = self.fc(x)
        return x

3. train the model to classification the ag_news

3.1 import kit and define some environment

In [8]:
import torch
import io
import time

from torch.optim import optimizer
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator, vocab
from torchtext.data.functional import to_map_style_dataset


train_file_path = "./train.csv"
test_file_path = "./test.csv"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ag_news_label = {1: "World",
                 2: "Sports",
                 3: "Business",
                 4: "Sci/Tec"}

# Hyper parameters
epoch = 10  # epoch
barch_size = 64  # batch size for training
LR = 5  # learning rate

3.2 define some function to prepare loading dataset or training the model

In [9]:
def yield_tokens(file_path):
    with io.open(file_path, encoding='utf-8') as f:
        for line in f:
            yield tokenizer(line)


def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)


def train(dataloader, loss_fn):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        optimizer.zero_grad()
        predicted_label = model(text, offsets)
        loss = loss_fn(predicted_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predicted_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()


def evaluate(dataloader, loss_fn):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predicted_label = model(text, offsets)
            loss = loss_fn(predicted_label, label)
            total_acc += (predicted_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [10]:
'''
1. Access to the raw dataset iterators and and Generate data batch and iterator
'''
train_iter = MyTextDataset(train_file_path, train=True)
test_iter = MyTextDataset(test_file_path, train=False)
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train], generator=torch.Generator().manual_seed(42))

train_dataloader = DataLoader(split_train_, batch_size=barch_size,
                              shuffle=True, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=barch_size,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=barch_size,
                             shuffle=True, collate_fn=collate_batch)

the train dataset are 120000's numbers
the test dataset are 7600's numbers


In [11]:
'''
2. Prepare data processing pipelines  
'''
tokenizer = get_tokenizer('basic_english')
vocab = build_vocab_from_iterator(yield_tokens(train_file_path), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])
# print(vocab["<unk>"])
# print(vocab(['here', 'is', 'the', 'an', 'example']))
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x) - 1

In [12]:
'''
3. Define the model
'''
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
em_size = 64
model = TextClassificationModel(vocab_size, em_size, num_class).to(device)

In [13]:
'''
4. Train the model
'''
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.1)
total_accu = None

for epoch in range(1, epoch + 1):
    epoch_start_time = time.time()
    train(train_dataloader, criterion)
    accu_val = evaluate(valid_dataloader, criterion)
    if total_accu is not None and total_accu > accu_val:
        scheduler.step()
    else:
        total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

| epoch   1 |   500/ 1782 batches | accuracy    0.393
| epoch   1 |  1000/ 1782 batches | accuracy    0.728
| epoch   1 |  1500/ 1782 batches | accuracy    0.810
-----------------------------------------------------------
| end of epoch   1 | time: 122.84s | valid accuracy    0.838 
-----------------------------------------------------------
| epoch   2 |   500/ 1782 batches | accuracy    0.859
| epoch   2 |  1000/ 1782 batches | accuracy    0.870
| epoch   2 |  1500/ 1782 batches | accuracy    0.876
-----------------------------------------------------------
| end of epoch   2 | time: 131.40s | valid accuracy    0.867 
-----------------------------------------------------------
| epoch   3 |   500/ 1782 batches | accuracy    0.896
| epoch   3 |  1000/ 1782 batches | accuracy    0.894
| epoch   3 |  1500/ 1782 batches | accuracy    0.900
-----------------------------------------------------------
| end of epoch   3 | time: 146.22s | valid accuracy    0.886 
----------------------------

4. test the model

In [14]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader, criterion)
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
test accuracy    0.906


5. you can predict your gossip!

In [20]:
def predict(text, text_pipeline):
    with torch.no_grad():
        text = torch.tensor(text_pipeline(text)).to(device)
        output = model(text, torch.tensor([0]).to(device))
        return output.argmax(1).item() + 1
    
ex_text_str1 = "Xiaoming like basketball!"
ex_text_str2 = "Wuhan University is one of best university!"
ex_text_str3 = "Wuhan is located in china!"

print(f"\'{ex_text_str1}\'  is a %s news" % ag_news_label[predict(ex_text_str1, text_pipeline)])
print(f"\'{ex_text_str2}\' is a %s news" % ag_news_label[predict(ex_text_str2, text_pipeline)])
print(f"\'{ex_text_str3}\' is a %s news" % ag_news_label[predict(ex_text_str3, text_pipeline)])

'Xiaoming like basketball!'  is a Sports news
'Wuhan University is one of best university!' is a Sci/Tec news
'Wuhan is located in china!' is a World news
